In [301]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import statsmodels.api as sm
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

전처리한 데이터가 모두 numerical data라서 df_actor의 데이터와 합치기 위해서는 

In [302]:

df_movies = pd.read_csv('[DS]TermProject_data.csv', encoding='cp949')


df_actors = pd.read_csv('df_actor.csv')
# 영화별 배우 리스트 생성
actor_grouped = df_actors.groupby('영화명')['배우'].apply(list).reset_index()


# 영화 데이터와 배우 데이터 병합
df_movies = pd.merge(df_movies, actor_grouped, on='영화명', how='left')

# 배우 데이터가 없는 영화 데이터 제거
df_movies = df_movies.dropna(subset=['배우'])

# '배우' 열의 값이 리스트가 아닌 경우 빈 리스트로 변환
df_movies['배우'] = df_movies['배우'].apply(lambda x: x if isinstance(x, list) else [])

# '배우' 열의 값 중 'float'형 데이터를 빈 리스트로 변환
df_movies['배우'] = df_movies['배우'].apply(lambda x: x if all(isinstance(i, str) for i in x) else [])





df_movies.info()
# 전국 매출액의 평균을 기준으로 흥행 여부(Y/N) 레이블 생성
mean_revenue = df_movies['전국 매출액'].mean()
df_movies['흥행'] = np.where(df_movies['전국 매출액'] > mean_revenue, 'Y', 'N')

df_movies.dropna(subset='배우',inplace=True)

# 데이터 확인
print(df_movies.head())

<class 'pandas.core.frame.DataFrame'>
Index: 851 entries, 0 to 2997
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영화명      851 non-null    object 
 1   감독       851 non-null    object 
 2   제작사      836 non-null    object 
 3   수입사      1 non-null      object 
 4   배급사      851 non-null    object 
 5   개봉일      851 non-null    object 
 6   영화유형     851 non-null    object 
 7   영화형태     851 non-null    object 
 8   국적       851 non-null    object 
 9   전국 스크린수  851 non-null    int64  
 10  전국 매출액   851 non-null    float64
 11  전국 관객수   851 non-null    int64  
 12  서울 매출액   851 non-null    int64  
 13  서울 관객수   851 non-null    int64  
 14  장르       851 non-null    object 
 15  등급       851 non-null    object 
 16  영화구분     851 non-null    object 
 17  배우       851 non-null    object 
dtypes: float64(1), int64(4), object(13)
memory usage: 126.3+ KB
          영화명   감독                              제작사  수입사  \
0       서울의 봄 

In [303]:

# 배우 리스트를 펼쳐서 데이터프레임으로 변환
actors = df_actors[['영화명', '배우']]

# 흥행 여부 데이터를 배우별로 병합
actors = actors.merge(df_movies[['영화명', '흥행']], on='영화명', how='left')

# 배우별 흥행 비율 계산
actor_success = actors.groupby('배우')['흥행'].value_counts(normalize=True).unstack().fillna(0)['Y']


#주연배우 흥행 비율 계산
def get_movie_average_actor_success_rate(data, actor_success_rate):
    average_success_rate = []
    for actor_string in data['배우']:
        
        actors = actor_string
        print(actors)
        success_rates = [actor_success_rate.get(actor, 0) for actor in actors]
        average_success_rate.append(np.mean(success_rates))
    return average_success_rate

df_movies['주연배우_흥행비율'] = get_movie_average_actor_success_rate(df_movies, actor_success)

# 데이터 확인
print(df_movies['주연배우_흥행비율'])

['황정민', '정우성', '이성민']
['최민식', '김고은', '유해진']
['마동석', '김무열', '박지환']
['김윤석', '백윤식', '정재영']
['라미란', '공명', '염혜란']
['류준열', '김태리', '김우빈']
['데이빗 필즈', '이호']
['손석구', '김성철', '김동휘']
['김해숙', '신민아', '강기영']
['설경구', '유준상', '진경']
['이동욱', '임수정', '이솜']
['윤여정', '유해진', '김윤진']
['나문희', '김영옥', '박근형']
['강하늘', '정소민']
['조진웅', '김희애', '이수경']
['엄상현', '박윤희', '강시현']
['김대중']
['최지우', '이유미', '최민호']
['신혜선', '이준영', '박정우']
['박성웅', '오대환', '오달수']
['강신일', '김규리', '백성현']
['김윤혜', '송이재', '박지훈']
['하정우', '임시완', '배성우']
['김은아', '엄상현', '신용우']
['슈가']
['김장하', '김주완']
['박혜수', '김시은']
['유선호', '강미나', '유인수']
['온유', '종현', '키']
['쟈니', '태용', '유타']
['김새해', '조경이', '황창영']
['오대환', '지승현', '이용규']
['김호중']
['김명곤']
['박기홍', '최경준', '안주민']
['이유영', '임선우', '노재원']
['박나은', '박효주', '김희원']
['주종혁', '문진승', '이주연']
['조병규', '한은수', '구준회']
['한해인', '이한주', '오만석']
['오동필', '오승준']
['김영성', '최준우', '이랑서']
['홍사빈', '송중기', '김형서']
['정이랑', '박성진', '이유준']
['정영선']
['박지연', '송지은']
['민시우', '안은미']
['타블로', '투컷', '미쓰라']
['한선화', '차미경', '한채아']
['유리', '길해연', '현우석']
['이순원', '서영주', '김수진']
['윤김명우',

C:\Users\leeminwoo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\leeminwoo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [304]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 851 entries, 0 to 2997
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   영화명        851 non-null    object 
 1   감독         851 non-null    object 
 2   제작사        836 non-null    object 
 3   수입사        1 non-null      object 
 4   배급사        851 non-null    object 
 5   개봉일        851 non-null    object 
 6   영화유형       851 non-null    object 
 7   영화형태       851 non-null    object 
 8   국적         851 non-null    object 
 9   전국 스크린수    851 non-null    int64  
 10  전국 매출액     851 non-null    float64
 11  전국 관객수     851 non-null    int64  
 12  서울 매출액     851 non-null    int64  
 13  서울 관객수     851 non-null    int64  
 14  장르         851 non-null    object 
 15  등급         851 non-null    object 
 16  영화구분       851 non-null    object 
 17  배우         851 non-null    object 
 18  흥행         851 non-null    object 
 19  주연배우_흥행비율  850 non-null    float64
dtypes: float64(2),

In [305]:
df_movies.drop(columns=['배우', '수입사', '개봉일', '영화유형','전국 매출액','전국 스크린수', '국적','서울 매출액','서울 관객수','영화형태'], axis=1, inplace=True)


director_rank = df_movies.groupby('감독')['전국 관객수'].mean().reset_index(name='director_rank').sort_values(by='director_rank', ascending=False)
director_rank['director_rank'] = [i+1 for i in range(director_rank.shape[0])]
df_movies = pd.merge(df_movies, director_rank, how='left')
df_movies.drop('감독', axis=1, inplace=True)
print(director_rank)


genre_rank = df_movies.groupby('장르')['전국 관객수'].mean().reset_index(name='gr_rank').sort_values(by='gr_rank', ascending=False)
genre_rank['gr_rank'] = [i+1 for i in range(genre_rank.shape[0])]
df_movies = pd.merge(df_movies, genre_rank, how='left')
df_movies.drop('장르', axis=1, inplace=True)
print(genre_rank)
distributor_rank = df_movies.groupby('배급사')['전국 관객수'].mean().reset_index(name='dis_rank').sort_values(by='dis_rank', ascending=False)
distributor_rank['dis_rank'] = [i+1 for i in range(distributor_rank.shape[0])]
df_movies = pd.merge(df_movies, distributor_rank, how='left')
df_movies.drop('배급사', axis=1, inplace=True)

print(distributor_rank)





      감독  director_rank
418  장재현              1
316  이상용              2
76   김성수              3
517  허명행              4
322  이석훈              5
..   ...            ...
326  이세원            539
381  이현준            540
504   하나            541
285  유희정            542
36   권유주            543

[543 rows x 2 columns]
          장르  gr_rank
10        범죄        1
15        액션        2
11        사극        3
9       미스터리        4
18       판타지        5
16      어드벤처        6
17       코미디        7
13       스릴러        8
6        드라마        9
4         기타       10
8        뮤지컬       11
7     멜로/로맨스       12
0         SF       13
14     애니메이션       14
3     공포(호러)       15
2         공연       16
5      다큐멘터리       17
1         가족       18
12  서부극(웨스턴)       19
                                 배급사  dis_rank
175      주식회사 에이비오엔터테인먼트,플러스엠 엔터테인먼트         1
128  롯데컬처웍스(주)롯데엔터테인먼트,(주)에이스메이커무비웍스         2
45                            (주)쇼박스         3
125                            다큐스토리         4
190          

In [306]:
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
for column in ['영화구분', '등급','제작사']:
    le = LabelEncoder()
    df_movies[column] = le.fit_transform(df_movies[column])
    label_encoders[column] = le

# 데이터 확인
print(df_movies['주연배우_흥행비율'])
df_movies.dropna(subset='주연배우_흥행비율',inplace=True)
df_movies.info()


0      0.644444
1      0.682540
2      0.866667
3      0.888889
4      0.277778
         ...   
846    0.000000
847    0.000000
848    0.000000
849    0.333333
850    0.000000
Name: 주연배우_흥행비율, Length: 851, dtype: float64
<class 'pandas.core.frame.DataFrame'>
Index: 850 entries, 0 to 850
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   영화명            850 non-null    object 
 1   제작사            850 non-null    int32  
 2   전국 관객수         850 non-null    int64  
 3   등급             850 non-null    int32  
 4   영화구분           850 non-null    int32  
 5   흥행             850 non-null    object 
 6   주연배우_흥행비율      850 non-null    float64
 7   director_rank  850 non-null    int64  
 8   gr_rank        850 non-null    int64  
 9   dis_rank       850 non-null    int64  
dtypes: float64(1), int32(3), int64(4), object(2)
memory usage: 63.1+ KB


In [307]:
# 필요한 피처 선택
features = ['영화구분', '등급', 'gr_rank', 'dis_rank', 'director_rank', '제작사', '주연배우_흥행비율'] 


# 피처와 타겟 분리
X = df_movies[features]
y = df_movies['전국 관객수']

# 결측치와 무한대 값 처리
X = X.replace([np.inf, -np.inf], np.nan).dropna()
y = y[X.index]  # X가 결측치를 삭제했기 때문에 y도 동일하게 맞춤

# 최적의 피처 조합 찾기
best_adj_r2 = -np.inf
best_features = None
best_feateures_list=[]
for i in range(2, len(features) + 1):  # 최소 세 개의 피처를 포함하는 조합부터 시작
    for combo in combinations(features, i):
        X_combo = X[list(combo)]
        X_train, X_test, y_train, y_test = train_test_split(X_combo, y, test_size=0.2, random_state=42)
        X_train = sm.add_constant(X_train)  # 상수항 추가
        model = sm.OLS(y_train, X_train)
        result = model.fit()
        p_values = result.pvalues[1:]  # 상수항 제외한 피처들의 p-value
        r_squared = result.rsquared
        print(f'Features: {combo}, Adjusted R^2: {result.rsquared_adj}, R^2: {r_squared}, p-values: {p_values.values}')
        if all(p_values.values<0.05):
            if result.rsquared_adj > best_adj_r2:
                best_adj_r2 = result.rsquared_adj
                best_features = combo

print(f'Best Adjusted R^2: {best_adj_r2}')
print(f'Best Feature Combination: {best_features}')

Features: ('영화구분', '등급'), Adjusted R^2: 0.06641607846204978, R^2: 0.06916595746510701, p-values: [1.81818409e-11 4.01851749e-02]
Features: ('영화구분', 'gr_rank'), Adjusted R^2: 0.08047874255036991, R^2: 0.08318719986244538, p-values: [3.50661224e-07 1.41822557e-04]
Features: ('영화구분', 'dis_rank'), Adjusted R^2: 0.08910171918107512, R^2: 0.09178477744563751, p-values: [2.11393634e-02 4.96241895e-06]
Features: ('영화구분', 'director_rank'), Adjusted R^2: 0.10057422925168591, R^2: 0.10322349514490625, p-values: [1.88555700e-02 5.80864625e-08]
Features: ('영화구분', '제작사'), Adjusted R^2: 0.06120694007511296, R^2: 0.06397216263748373, p-values: [4.21653885e-11 5.04309191e-01]
Features: ('영화구분', '주연배우_흥행비율'), Adjusted R^2: 0.19659175273306007, R^2: 0.1989581982331099, p-values: [9.65435082e-01 8.15796305e-25]
Features: ('등급', 'gr_rank'), Adjusted R^2: 0.04599848295881892, R^2: 0.04880850215481647, p-values: [3.06667741e-01 3.30168101e-08]
Features: ('등급', 'dis_rank'), Adjusted R^2: 0.08455962649794713, 

In [310]:

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold

# 최적의 피처 조합으로 데이터 분할
X_best = X[list(best_features)]
y = df_movies['흥행']
X_train, X_test, y_train, y_test = train_test_split(X_best, y, test_size=0.2, random_state=42)


# 나이브 베이즈 분류기 학습 및 K-fold 교차 검증
nb = GaussianNB()
kf = KFold(n_splits=10, shuffle=True, random_state=42)
# 교차 검증 점수 계산
cv_scores = cross_val_score(nb, X_best, y, cv=kf, scoring='accuracy')
print(f'K-fold Cross Validation Scores: {cv_scores}')
print(f'Mean CV Accuracy: {np.mean(cv_scores)}')


# 교차 검증 예측값 계산
y_pred_cv = cross_val_predict(nb, X_best, y, cv=kf)

# 혼동 행렬 및 분류 보고서 출력
conf_matrix = confusion_matrix(y, y_pred_cv)
class_report = classification_report(y, y_pred_cv)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

print("bagging")
# 배깅 분류기 생성
bagging_nb = BaggingClassifier(estimator=nb, n_estimators=10, random_state=42)

# 모델 학습
bagging_nb.fit(X_train, y_train)

# 예측
y_pred = bagging_nb.predict(X_test)

# 전체예측
y_pred_full = bagging_nb.predict(X_best)

# 혼동 행렬 및 분류 보고서
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", cr)

# 예측 결과와 실제 값 출력
#comparison_df = pd.DataFrame({ '실제 값': y_test.values, '예측 결과': y_pred})

# 전체 예측 결과와 실제 값 출력
comparison_df_full = pd.DataFrame({'실제 값': y.values, '예측 결과': y_pred_full})
comparison_df_full = pd.concat([comparison_df_full,df_movies['영화명']], axis=1)
comparison_df_full.to_csv("C:/Users/leeminwoo/Desktop/데과팀플/prediction_bagging.csv")
# 예측 성공과 실패 수
print(comparison_df_full['실제 값'].value_counts())
print(comparison_df_full['예측 결과'].value_counts())


# 혼동 행렬 계산
tn, fp, fn, tp = cm.ravel()

# 정확도 계산
accuracy = (tp + tn) / (tp + tn + fp + fn)

# 민감도 계산
sensitivity = tp / (tp + fn)

# 특이도 계산
specificity = tn / (tn + fp)

# 결과 출력
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
# 나이브 베이즈 분류기 생성
nb = GaussianNB()
print("adaboost")
# 부스팅 분류기 생성
adaboost_nb = AdaBoostClassifier(estimator=nb, n_estimators=10, random_state=42)

# 모델 학습
adaboost_nb.fit(X_train, y_train)

# 예측
y_pred = adaboost_nb.predict(X_test)

# 혼동 행렬 및 분류 보고서
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", cr)
# 혼동 행렬 계산
tn, fp, fn, tp = cm.ravel()

# 정확도 계산
accuracy = (tp + tn) / (tp + tn + fp + fn)

# 민감도 계산
sensitivity = tp / (tp + fn)

# 특이도 계산
specificity = tn / (tn + fp)

# 결과 출력
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

# 전체예측
y_pred_full = adaboost_nb.predict(X_best)
# 전체 예측 결과와 실제 값 출력
comparison_df_full = pd.DataFrame({'실제 값': y.values, '예측 결과': y_pred_full})
comparison_df_full = pd.concat([comparison_df_full,df_movies['영화명']], axis=1)
comparison_df_full.to_csv("C:/Users/leeminwoo/Desktop/데과팀플/prediction_adaboost.csv")
# 예측 성공과 실패 수
print(comparison_df_full['실제 값'].value_counts())
print(comparison_df_full['예측 결과'].value_counts())

K-fold Cross Validation Scores: [0.96470588 0.94117647 0.88235294 0.90588235 0.96470588 0.89411765
 0.94117647 0.85882353 0.96470588 0.94117647]
Mean CV Accuracy: 0.9258823529411764
Confusion Matrix:
 [[703  48]
 [ 15  84]]
Classification Report:
               precision    recall  f1-score   support

           N       0.98      0.94      0.96       751
           Y       0.64      0.85      0.73        99

    accuracy                           0.93       850
   macro avg       0.81      0.89      0.84       850
weighted avg       0.94      0.93      0.93       850

bagging
Confusion Matrix:
 [[150   6]
 [  2  12]]

Classification Report:
               precision    recall  f1-score   support

           N       0.99      0.96      0.97       156
           Y       0.67      0.86      0.75        14

    accuracy                           0.95       170
   macro avg       0.83      0.91      0.86       170
weighted avg       0.96      0.95      0.96       170

실제 값
N    751
Y     99


C:\Users\leeminwoo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
